In [1]:
import eranest

In [4]:
# eranest.era5ify_geojson(request_id, variables, start_date, end_date, json_file, frequency, resolution)
# request_id : str, unique identifier for the request
# variables : list, list of variables to download
# start_date : string, start date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# end_date : string, end date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# json_file : str, path to the geojson file containing the area of interest
# dataset_type : str, type of dataset (single or pressure, for single level or pressure level datasets), optional (single by default)
# pressure_levels : list, list of strings of pressure levels to download (e.g., ["1000", "925", "850"]), optional (empty by default)
# frequency : str, frequency of the data (hourly, daily, weekly, monthly, yearly), optional (hourly by default)
# resolution : float, resolution of the data in degrees (0.1, 0.25, etc.), optional (0.25 by default)

In [2]:
df = eranest.era5ify_geojson(
    request_id="test",
    variables=[
        "2m_temperature",
        "total_precipitation",
        "surface_pressure",
        "2m_dewpoint_temperature",
    ],
    start_date="2023-1-1",
    end_date="2023-1-31", 
    json_file="../data/india.json",
    dataset_type="single",
    frequency="daily",
    resolution="0.25",
)

Successfully loaded JSON file with utf-8 encoding
✓ CDS API configuration is already set up and valid.

STARTING ERA5 SINGLE LEVEL PROCESSING
Request ID: test
Variables: ['2m_temperature', 'total_precipitation', 'surface_pressure', '2m_dewpoint_temperature']
Date Range: 2023-01-01 to 2023-01-31
Frequency: daily
Resolution: 0.25°
GeoJSON File: C:\Users\ATHARV~1\AppData\Local\Temp\test_temp_geojson.json
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 35.4940°
  South: 7.9655°
  East:  97.4026°
  West:  68.1766°
  Area:  29.2259° × 27.5285°


--- GeoJSON Mini Map ---

MINI MAP (68.18°W to 97.40°E, 7.97°S to 35.49°N):
┌─────────────────────────────────────────┐
│·········································│
│········■■■■■■■··························│
│··········■■■■■··························│
│·········■■■■■■■·························│
│·······■■■■■■■■■■························│
│····■■■■■■■■■■■■■■■■········■····■■■■■■■·│
│···■■■■■■■■■■■■■■■■■■■■

2025-07-09 23:37:03,723 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-09 23:37:05,515 INFO Request ID is 3a4a106c-b140-41c4-94fd-7ae8f57bc174
2025-07-09 23:37:05,687 INFO status has been updated to accepted
2025-07-09 23:37:14,593 INFO status has been updated to successful


  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk1.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk1.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk1\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk1\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 111, 'longitude': 117})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 111, 'longitude': 117})
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.46 seconds
  - Points inside: 4446
  - Points outside: 8541
  - Percentage inside:

2025-07-09 23:37:39,011 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-09 23:37:39,730 INFO Request ID is 1844955e-fcc0-4d1a-86ea-d6d5f82d81a9
2025-07-09 23:37:39,873 INFO status has been updated to accepted
2025-07-09 23:37:45,124 INFO status has been updated to running
2025-07-09 23:37:48,659 INFO status has been updated to successful


  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk2.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk2.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk2\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk2\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 111, 'longitude': 117})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 111, 'longitude': 117})
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.49 seconds
  - Points inside: 4446
  - Points outside: 8541
  - Percentage inside:

2025-07-09 23:38:10,490 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-09 23:38:11,138 INFO Request ID is c19073af-a29d-431e-a6e5-0d7064724258
2025-07-09 23:38:11,306 INFO status has been updated to accepted
2025-07-09 23:38:20,358 INFO status has been updated to running
2025-07-09 23:38:25,587 INFO status has been updated to successful


  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk3.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk3.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk3\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk3\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 72, 'latitude': 111, 'longitude': 117})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 72, 'latitude': 111, 'longitude': 117})
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.46 seconds
  - Points inside: 4446
  - Points outside: 8541
  - Percentage inside: 3

In [3]:
df = eranest.era5ify_geojson(
    request_id="test2",
    variables=[
        "2m_temperature", 
        "total_precipitation"
    ],
    start_date="2024-1-1",
    end_date="2024-12-31",
    json_file="../data/latvia.geojson",
    dataset_type="single",
    frequency="monthly",
    resolution="0.1",
)

Successfully loaded JSON file with utf-8 encoding
✓ CDS API configuration is already set up and valid.

STARTING ERA5 SINGLE LEVEL PROCESSING
Request ID: test2
Variables: ['2m_temperature', 'total_precipitation']
Date Range: 2024-01-01 to 2024-12-31
Frequency: monthly
Resolution: 0.1°
GeoJSON File: C:\Users\ATHARV~1\AppData\Local\Temp\test2_temp_geojson.json
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 58.0856°
  South: 55.6776°
  East:  28.2431°
  West:  20.9537°
  Area:  7.2894° × 2.4080°


--- GeoJSON Mini Map ---

MINI MAP (20.95°W to 28.24°E, 55.68°S to 58.09°N):
┌────────────────────────────────────────────────────────────┐
│····························································│
│····························■■■■■■■■■■■■■■··················│
│······■■■■■■■■■■············■■■■■■■■■■■■■■■■■■■■■■■■■■······│
│····■■■■■■■■■■■■■■■········■■■■■■■■■■■■■■■■■■■■■■■■■■■■■····│
│··■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■·····

2025-07-09 23:40:52,038 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-09 23:40:52,924 INFO Request ID is 44426d8c-9ec2-46de-829a-faea62a42596
2025-07-09 23:40:53,075 INFO status has been updated to accepted
2025-07-09 23:41:07,548 INFO status has been updated to successful


  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test2.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test2.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test2\data_stream-moda_stepType-avgad.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test2\data_stream-moda_stepType-avgua.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-moda_stepType-avgad.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 12, 'latitude': 25, 'longitude': 73})
  Processing file 2/2: data_stream-moda_stepType-avgua.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 12, 'latitude': 25, 'longitude': 73})
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 1825 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.36 seconds
  - Points inside: 962
  - Points outside: 863
  - Percentage inside: 52.71%
→ Filtering original dataset 

In [ ]:
# eranest.era5ify_bbox(request_id, variables, start_date, end_date, north, south, east, west, frequency, resolution)
# request_id : str, unique identifier for the request
# variables : list, list of variables to download
# start_date : string, start date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# end_date : string, end date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# north : float, northern boundary of the bounding box
# south : float, southern boundary of the bounding box
# east : float, eastern boundary of the bounding box
# west : float, western boundary of the bounding box
# dataset_type : str, type of dataset (single or pressure, for single level or pressure level datasets), optional (single by default)
# pressure_levels : list, list of strings of pressure levels to download (e.g., ["1000", "925", "850"]), optional (empty by default)
# frequency : str, frequency of the data (hourly, daily, weekly, monthly, yearly), optional (hourly by default)
# resolution : float, resolution of the data in degrees (0.1, 0.25, etc.), optional (0.25 by default)

In [4]:
df = eranest.era5ify_bbox(
    request_id="test3",
    variables=[
        "2m_temperature", 
        "total_precipitation"
    ],
    start_date="2024-01-1",
    end_date="2024-01-15",
    north = 30,
    south = 20,
    east = 80,
    west = 70,
    frequency="daily",
    resolution=0.25
)

✓ CDS API configuration is already set up and valid.

STARTING ERA5 SINGLE LEVEL PROCESSING
Request ID: test3
Variables: ['2m_temperature', 'total_precipitation']
Date Range: 2024-01-01 to 2024-01-15
Frequency: daily
Resolution: 0.25°
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 30.0000°
  South: 20.0000°
  East:  80.0000°
  West:  70.0000°
  Area:  10.0000° × 10.0000°

--- Processing Strategy ---
Using monthly dataset: False
Total days to process: 15
Max days per chunk: 14
Needs chunking: True
PROCESSING CHUNK 1/2
Date Range: 2024-01-01 to 2024-01-14
Variables:  2m_temperature, total_precipitation
  → Downloading ERA5 data (attempt 1/6)...


2025-07-09 23:43:45,653 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-09 23:43:46,673 INFO Request ID is d09516ab-bdf8-4921-b3b2-f37d9cc95de8
2025-07-09 23:43:46,850 INFO status has been updated to accepted
2025-07-09 23:44:01,072 INFO status has been updated to successful


  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk1.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk1.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk1\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk1\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 41, 'longitude': 41})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 41, 'longitude': 41})
✓ Chunk completed in 19.9 seconds
PROCESSING CHUNK 2/2
Date Range: 2024-01-15 to 2024-01-15
Variables:  2m_temperature, total_precipitation
  → Downloading ERA5 data (attempt 1/6)...


2025-07-09 23:44:15,555 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-09 23:44:16,186 INFO Request ID is 28bc17d1-7ec5-457f-b25e-de8b1882035f
2025-07-09 23:44:16,342 INFO status has been updated to accepted
2025-07-09 23:44:25,060 INFO status has been updated to successful


  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk2.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk2.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk2\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk2\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 24, 'latitude': 41, 'longitude': 41})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 24, 'latitude': 41, 'longitude': 41})
✓ Chunk completed in 12.3 seconds
AGGREGATING DATA (DAILY)
Aggregating data to daily frequency...
Sum columns: ['tp']
Max columns: []
Min columns: []
Rate columns: []
Average columns: ['t2m']
Aggregation completed in:   0.12 seconds

Saving files to output directory: test3_output
  Saved final data to: test3_

In [5]:
df = eranest.era5ify_bbox(
    request_id="test4",
    variables=[
        "temperature", 
        "relative_humidity"
    ],
    start_date="2024-01-1",
    end_date="2024-01-15",
    north = 30,
    south = 20,
    east = 80,
    west = 70,
    dataset_type="pressure",
    pressure_levels=["1000"],
    frequency="daily",
    resolution=0.25
)

✓ CDS API configuration is already set up and valid.

STARTING ERA5 PRESSURE LEVEL PROCESSING
Request ID: test4
Variables: ['temperature', 'relative_humidity']
Pressure Levels: ['1000']
Date Range: 2024-01-01 to 2024-01-15
Frequency: daily
Resolution: 0.25°
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 30.0000°
  South: 20.0000°
  East:  80.0000°
  West:  70.0000°
  Area:  10.0000° × 10.0000°

--- Processing Strategy ---
Using monthly dataset: False
Total days to process: 15
Max days per chunk: 14
Needs chunking: True
PROCESSING CHUNK 1/2
Date Range: 2024-01-01 to 2024-01-14
Variables:  temperature, relative_humidity
Levels:     1000
  → Downloading ERA5 data (attempt 1/6)...


2025-07-10 00:01:26,904 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-10 00:01:27,295 INFO Request ID is 87781187-5ec5-4cb3-b3d7-8a5660d4d23e
2025-07-10 00:01:27,516 INFO status has been updated to accepted
2025-07-10 00:01:36,353 INFO status has been updated to successful


  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk1.nc
Copying NetCDF file: C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk1.nc
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk1\test4_chunk1.nc

Processing downloaded data:
- Found 1 file(s)
  Processing file 1/1: test4_chunk1.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'pressure_level': 1, 'latitude': 41, 'longitude': 41})
✓ Chunk completed in 13.3 seconds
PROCESSING CHUNK 2/2
Date Range: 2024-01-15 to 2024-01-15
Variables:  temperature, relative_humidity
Levels:     1000
  → Downloading ERA5 data (attempt 1/6)...


2025-07-10 00:01:50,159 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-10 00:01:50,531 INFO Request ID is 7a01950c-e074-4377-96f3-6ecf468fd7be
2025-07-10 00:01:50,675 INFO status has been updated to accepted
2025-07-10 00:01:59,388 INFO status has been updated to running
2025-07-10 00:02:04,591 INFO status has been updated to successful


  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk2.nc
Copying NetCDF file: C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk2.nc
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk2\test4_chunk2.nc

Processing downloaded data:
- Found 1 file(s)
  Processing file 1/1: test4_chunk2.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 24, 'pressure_level': 1, 'latitude': 41, 'longitude': 41})
✓ Chunk completed in 17.0 seconds
AGGREGATING DATA (DAILY)
Aggregating pressure level data to daily frequency...
Variables to average: ['t', 'r']
Including pressure_level in aggregation groups
Aggregation completed in:   0.06 seconds

Saving files to output directory: test4_output
  Saved final data to: test4_output\test4_daily_data.csv
  Saved unique coordinates to: test4_output\test4_unique_latlongs.csv
  Saved raw data to: test4_output\test4_raw_data.csv

PROCESSING COMPLETE

RESULTS SUMMARY:
----------------------------------------
Variables processed: 2

In [ ]:
# eranest.search_variable(pattern, dataset_type)
# pattern : str, pattern to search for in variable names from dataset in dataset_type
# dataset_type :str,  Dataset type to search ("single", "pressure", "all", or any other registered dataset, all by dedault)

In [ ]:
eranest.search_variable("teMp ", dataset_type="all")


=== SEARCH RESULTS (ALL LEVELS) ===
Pattern: 'temp'
Variables found: 21

--- Temperature And Pressure ---
1. 2m_dewpoint_temperature (from single levels)
   Description: Dew point temperature at 2 meters above the surface, representing the temperature at which air becomes saturated with moisture. Unit: Kelvin (K).

2. 2m_temperature (from single levels)
   Description: Air temperature at 2 meters above the surface, typically used to represent surface-level weather conditions. Unit: Kelvin (K).

3. ice_temperature_layer_1 (from single levels)
   Description: Temperature of the top layer of sea ice. This layer is most affected by atmospheric conditions. Unit: Kelvin (K).

4. ice_temperature_layer_2 (from single levels)
   Description: Temperature of the second layer of sea ice, representing deeper internal ice temperatures. Unit: Kelvin (K).

5. ice_temperature_layer_3 (from single levels)
   Description: Temperature of the third layer of sea ice, indicating mid-level internal ice tempe

In [ ]:
# eranest.describe_variables(variable_names, dataset_type)
# variable_names : list, list of variable names to describe
# dataset_type : str, type of dataset (single or pressure, for single level or pressure level datasets)

In [ ]:
eranest.describe_variables(
    variable_names=["2m_temperature", "total_precipitation", "surface_pressure"], dataset_type="single"
)


=== Variable Descriptions (SINGLE LEVELS) ===

2m_temperature:
  Category: temperature_and_pressure
  Description: Air temperature at 2 meters above the surface, typically used to represent surface-level weather conditions. Unit: Kelvin (K).

total_precipitation:
  Category: precipitation_variables
  Description: Cumulative precipitation (convective + large-scale). Unit: meters (m) of water equivalent.

surface_pressure:
  Category: temperature_and_pressure
  Description: Atmospheric pressure at the surface of the Earth. Influenced by elevation and weather systems. Unit: Pascal (Pa).
